In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

# Model: logistic regression

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, roc_auc_score, roc_curve,  precision_recall_curve
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

## Data

Load the dataset, applying no major transformations to it.

In [ ]:
data = pd.read_csv('../dataset/creditcard.csv')
data.head()

In [ ]:
X = data.drop(columns=['Class'])
y = data['Class']

Since the data is largely unbalanced we must use a stratified sampling to make sure we get both negative and positive samples to train with.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

## Pipeline (build)

In [ ]:
numeric_feature_indexes = slice(0, 30)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
])

In [ ]:
num_features_type_map = {feature: 'float64' for feature in X_train.columns[numeric_feature_indexes]}

X_train = X_train.astype(num_features_type_map)
X_test = X_test.astype(num_features_type_map)

In [ ]:
pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)

## Pipeline (train)

In [ ]:
model = pipeline.fit(X_train, y_train)
model 

## Pipeline (evaluate)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
disp = plot_confusion_matrix(model, X_test, y_test, display_labels=['normal', 'fraudulent'], cmap=plt.cm.Blues)
disp.ax_.grid(False)

Some great material is available here: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

In [ ]:
y_pred_proba = pipeline.predict_proba(X_test)[::,1]

fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(fpr,tpr,label=f"auc {auc:2.2f}")
ax.legend(loc=4)

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate');

In [ ]:
precision, recall, _ = precision_recall_curve(y_test,  y_pred_proba)
fig, ax = plt.subplots(figsize=(5,5))

no_skill = len(y_test[y_test==1]) / len(y_test)
ax.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
ax.plot(recall, precision)

ax.set_xlabel('Precision')
ax.set_ylabel('Recall');